In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

import keras
import keras.backend
from keras import layers
from keras import models
from keras import optimizers
import keras.utils

print(tf.__version__)

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
#import math
#import time
#import h5py
import sklearn
from sklearn.utils import shuffle

import os

/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/schubert/.conda/envs/LRP/lib/python3.7/site-packages

1.13.1


## Useful Functions

In [ ]:
def build_model():
    input2=[layers.Input(shape=(len(X_train[i][0]),)) for i in range(1,len(X_train))]
    input1 = layers.Input(shape = (grid, grid,1))
    x = layers.Conv2D(32, (5, 5), activation='relu',padding='same')(input1)
    x = layers.Conv2D(32, (3, 3), activation='relu',padding='same')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu',padding='same')(x)
    x = layers.MaxPool2D((2, 2))(x)
    x1 = layers.Flatten()(x)
    x = layers.concatenate(inputs = [x1]+input2,axis=-1)
    x = layers.Dense(64, activation='relu')(x)
    output = layers.Dense(2, activation='softmax')(x)
    model = models.Model(inputs=[input1] +input2, outputs=output)
    opt=keras.optimizers.Adam(lr=0.0005,beta_1=0.9, beta_2=0.9, amsgrad=False)
    model.compile(loss='binary_crossentropy',
                optimizer=opt,
                metrics=['binary_crossentropy', 'accuracy'])
    return model

In [ ]:
def build_XY(features,label,dic):
    X_test=[dic[key] for key in features]
    Y_test=[dic[key] for key in label]
    X_test[0]=X_test[0].reshape(n_test, grid, grid, 1)
    dim=[1 if ele[0].shape==() else ele[0].shape[0] for ele in X_test]
    for i in range(1,len(X_test)):
        X_test[i]=X_test[i].reshape(len(X_test[i]),dim[i])
    return X_test,Y_test

## Import data:
Using Train and Test set produced by preprocessing notebook.

In [2]:
grid = 16
data_train= np.load('data/ShowJet_full_train.npz')

In [3]:
n_train=len(data_train['jetPt'])
print(n_train)

893412
99270


In [4]:
feat_all=[key for key in data_train.keys()];feat_all.remove('labels');feat_all

['jetImages',
 'jetPt',
 'jetEta',
 'jetPhi',
 'tau21',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult',
 'jetpull',
 'Abs_jetpull']

# Build Model with all Features

### Select Features for Model

Show all possible features

In [5]:
# features will be loaded into X
features=['jetImages',
 'jetPt',
 'jetEta',
 'jetPhi',
 'tau21',
 'chMult',
 'neutMult',
 'phoMult',
 'eleMult',
 'muMult',
 'jetpull']
# label into Y
label=['labels']

In [ ]:
X_train,Y_train=build_XY(features,label,data_train)

### Build DNN:

Initialize model:

In [26]:
CNN = build_model()

Test model's prediction $before$ training:

In [27]:
X_batch =[ele[:10] for ele in X_train]
Y_batch =[ele[:10] for ele in Y_train]
example_result = CNN.predict(x = X_batch)
results = CNN.evaluate(x = X_batch, y =Y_batch )
print(example_result)
print(results)

10/10 [==============================] - 0s 9ms/step
[[0.52069694 0.47930306]
 [0.5344897  0.46551028]
 [0.5261534  0.47384658]
 [0.52999145 0.47000852]
 [0.5235912  0.4764088 ]
 [0.5062274  0.49377266]
 [0.5210249  0.47897512]
 [0.5147497  0.4852503 ]
 [0.5179273  0.48207277]
 [0.52657217 0.47342786]]
[0.6499219536781311, 0.6499219536781311, 1.0]


Now train! (warning: if building CNN, computer tends to get loud)

In [30]:
checkpoint_path = "model/CNN_all.h1"
if not os.path.exists("model"):
    os.mkdir("model")

model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', 
                                   verbose=1, save_best_only=True, 
                                   save_weights_only=False, mode='auto', 
                                   period=1)    
EPOCHS = 4
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = CNN.fit(
  X_train, Y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose = 1,
  callbacks=[early_stop, model_checkpoint])

Instructions for updating:
Use tf.cast instead.
Train on 714729 samples, validate on 178683 samples
Epoch 1/4
714729/714729 [==============================] - 1109s 2ms/step - loss: 0.4144 - binary_crossentropy: 0.4144 - acc: 0.8201 - val_loss: 0.4972 - val_binary_crossentropy: 0.4972 - val_acc: 0.7769

Epoch 00001: val_loss improved from inf to 0.49717, saving model to model/CNN_all.h1
Epoch 2/4
714729/714729 [==============================] - 1083s 2ms/step - loss: 0.3899 - binary_crossentropy: 0.3899 - acc: 0.8363 - val_loss: 0.5700 - val_binary_crossentropy: 0.5700 - val_acc: 0.7404

Epoch 00002: val_loss did not improve from 0.49717
Epoch 3/4
714729/714729 [==============================] - 1094s 2ms/step - loss: 0.3852 - binary_crossentropy: 0.3852 - acc: 0.8393 - val_loss: 0.5001 - val_binary_crossentropy: 0.5001 - val_acc: 0.7775

Epoch 00003: val_loss did not improve from 0.49717
Epoch 4/4
714729/714729 [==============================] - 1265s 2ms/step - loss: 0.3828 - binary_

# Build 10 Models with one Expert and fit with subset of features

In [31]:
for ii in range(1,len(feat_all)-1):
    # Select features and load into X and Y
    features=['jetImages']+[feat_all[ii]]
    label=['labels']
    X_train,Y_train=build_XY(features,label,data_train)
    # Build Model
    circleCNN = build_model()
    # Fit 
    checkpoint_path = "model/CNN_sub_"+str(ii)+".h1"
    if not os.path.exists("model"):
        os.mkdir("model")
    model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', 
                                   verbose=1, save_best_only=True, 
                                   save_weights_only=False, mode='auto', 
                                   period=1)    
    EPOCHS = 3
    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
    history = circleCNN.fit(
      X_train, Y_train,
      epochs=EPOCHS, validation_split = 0.2, verbose = 1,
      callbacks=[early_stop, model_checkpoint])
    circleCNN.save(checkpoint_path)

Train on 714729 samples, validate on 178683 samples
Epoch 1/3
714729/714729 [==============================] - 760s 1ms/step - loss: 0.4989 - binary_crossentropy: 0.4989 - acc: 0.7595 - val_loss: 0.6870 - val_binary_crossentropy: 0.6870 - val_acc: 0.6358

Epoch 00001: val_loss improved from inf to 0.68697, saving model to model/CNN_sub_1.h1
Epoch 2/3
714729/714729 [==============================] - 640s 896us/step - loss: 0.4674 - binary_crossentropy: 0.4674 - acc: 0.7827 - val_loss: 0.7175 - val_binary_crossentropy: 0.7175 - val_acc: 0.6324

Epoch 00002: val_loss did not improve from 0.68697
Epoch 3/3
714729/714729 [==============================] - 652s 913us/step - loss: 0.4627 - binary_crossentropy: 0.4627 - acc: 0.7861 - val_loss: 0.7016 - val_binary_crossentropy: 0.7016 - val_acc: 0.6107

Epoch 00003: val_loss did not improve from 0.68697
Train on 714729 samples, validate on 178683 samples
Epoch 1/3
714729/714729 [==============================] - 668s 934us/step - loss: 0.5322 -


Epoch 00001: val_loss improved from inf to 0.82187, saving model to model/CNN_sub_10.h1
Epoch 2/3
714729/714729 [==============================] - 502s 703us/step - loss: 0.5083 - binary_crossentropy: 0.5083 - acc: 0.7642 - val_loss: 0.7133 - val_binary_crossentropy: 0.7133 - val_acc: 0.6051

Epoch 00002: val_loss improved from 0.82187 to 0.71331, saving model to model/CNN_sub_10.h1
Epoch 3/3
714729/714729 [==============================] - 502s 703us/step - loss: 0.5021 - binary_crossentropy: 0.5021 - acc: 0.7690 - val_loss: 0.6800 - val_binary_crossentropy: 0.6800 - val_acc: 0.6306

Epoch 00003: val_loss improved from 0.71331 to 0.67998, saving model to model/CNN_sub_10.h1


### Fit Absolute Jet_pull seperately
This was added after other Variabels

In [ ]:
# Select features and load into X and Y
features=['jetImages']+[feat_all[11]]
label=['labels']
X_train,Y_train=build_XY(features,label,dic)
# Build Model
circleCNN = build_model()
# Fit 
checkpoint_path = "model/CNN_sub_"+str(11)+".h1"
if not os.path.exists("model"):
    os.mkdir("model")
model_checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', 
                               verbose=1, save_best_only=True, 
                               save_weights_only=False, mode='auto', 
                               period=1)    
EPOCHS = 20
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
history = circleCNN.fit(
  X_train, Y_train,
  epochs=EPOCHS, validation_split = 0.2, verbose = 1,
  callbacks=[early_stop, model_checkpoint])
circleCNN.save(checkpoint_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 714729 samples, validate on 178683 samples
Epoch 1/20
714729/714729 [==============================] - 586s 820us/step - loss: 0.5261 - binary_crossentropy: 0.5261 - acc: 0.7493 - val_loss: 0.7375 - val_binary_crossentropy: 0.7375 - val_acc: 0.5732

Epoch 00001: val_loss improved from inf to 0.73754, saving model to model/CNN_sub_11.h1
Epoch 2/20
714729/714729 [==============================] - 599s 838us/step - loss: 0.5016 - binary_crossentropy: 0.5016 - acc: 0.7685 - val_loss: 0.7052 - val_binary_crossentropy: 0.7052 - val_acc: 0.6052

Epoch 00002: val_loss improved from 0.73754 to 0.70521, saving model to model/CNN_sub_11.h1
Epoch 3/20
564864/714729 [======================>.......] - ETA: 2:07 - loss: 0.4994 - binary_crossentropy: 0.4994 - acc: 0.7697